# CCLE Raw Gene Expression Data Importation
**Local Version**: 1
**Source Version**: NA

This notebook will import raw CCLE gene expression data through the [CGDS](http://www.cbioportal.org/cgds_r.jsp) (aka "Cancer Genomic Data Server" portal.  This should not be confused with the [GDSC](http://www.cancerrxgene.org/) portal which is a separate data source entirely.

In [2]:
%run -m ipy_startup
%run -m ipy_logging
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import api
from mgds.data_aggregation.import_lib import ccle
from mgds.data_aggregation.import_lib import cgds
pd.set_option('display.max_info_rows', 25000000)

In [3]:
case_list_id = ccle.CASE_LIST_ID
genetic_profile_id = ccle.PROF_GENE_EXPRESSION
batch_size = 50

op = lambda: cgds.get_genetic_profile_data(
    case_list_id, genetic_profile_id,
    api.get_hugo_gene_ids(), gene_id_batch_size=batch_size
)
d = db.cache_raw_operation(op, src.CCLE_v1, 'gene-expression')

2016-11-18 12:04:21,265:DEBUG:mgds.data_aggregation.io_utils: Restoring serialized object from "/Users/eczech/data/research/mgds/raw/ccle_v1_gene-expression.pkl"


In [4]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39032 entries, 0 to 7
Columns: 1021 entries, GENE_ID to UOK101_KIDNEY
dtypes: float64(1019), int64(1), object(1)
memory usage: 304.3+ MB


In [5]:
d = cgds.melt_raw_data(d)
d.info()

[Remove null values for column "VALUE"] Records before = 39773608, Records after = 15531954, Records removed = 24241654 (%60.95)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15531954 entries, 0 to 39149095
Data columns (total 4 columns):
GENE_ID:CGDS    15531954 non-null int64
GENE_ID:HGNC    15531954 non-null object
CELL_LINE_ID    15531954 non-null object
VALUE           15531954 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 592.5+ MB


In [6]:
d_agg, d_dist = cgds.aggregate(d)
d_agg.head()

,CELL_LINE_ID,GENE_ID:HGNC,GENE_ID:CGDS,VALUE_MEAN,VALUE_STD
0,1321N1_CENTRAL_NERVOUS_SYSTEM,A1BG,1,-0.1004,0.0
1,1321N1_CENTRAL_NERVOUS_SYSTEM,A1CF,29974,-0.4882,0.0
2,1321N1_CENTRAL_NERVOUS_SYSTEM,A2M,2,-0.2826,0.0
3,1321N1_CENTRAL_NERVOUS_SYSTEM,A2M-AS1,144571,-0.7492,0.0
4,1321N1_CENTRAL_NERVOUS_SYSTEM,A2ML1,144568,0.0017,0.0


In [7]:
d_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15512614 entries, 0 to 15512613
Data columns (total 5 columns):
CELL_LINE_ID    15512614 non-null object
GENE_ID:HGNC    15512614 non-null object
GENE_ID:CGDS    15512614 non-null int64
VALUE_MEAN      15512614 non-null float64
VALUE_STD       15512614 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 591.8+ MB


In [8]:
d_dist

1    15493274
2       19340
Name: Number of Replicates, dtype: int64

In [11]:
assert np.all(pd.notnull(d_agg))
db.save(d, src.CCLE_v1, db.IMPORT, 'gene-expression')

'/Users/eczech/data/research/mgds/import/ccle_v1_gene-expression.pkl'